In [6]:
!pip install dask hvplot --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 5.1 MB/s eta 0:00:00


In [7]:
#importing Libraries
from dask.distributed import Client
import dask.dataframe as dd
import hvplot.dask
import pandas as pd


In [8]:
# starting a local dask client
client = Client()
client  # shows dashboard link


INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:46165
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:45201'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36701'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:37139 name: 0
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:37139
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48468
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:34231 name: 1
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:34231
INFO:distributed.core:Starting established connection to tcp://127

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 12.67 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46165,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:37139,Total threads: 1
Dashboard: http://127.0.0.1:40139/status,Memory: 6.34 GiB
Nanny: tcp://127.0.0.1:45201,


In [ ]:
from google.colab import files

uploaded = files.upload()


Saving flights.csv to flights.csv


In [10]:
# Adding dataset path
import dask.dataframe as dd

# Define dtypes explicitly for problematic columns
dtype_fix = {
    "AIR_TIME": "float64",
    "ARRIVAL_DELAY": "float64",
    "ARRIVAL_TIME": "float64",
    "CANCELLATION_REASON": "object",  # keep as string
    "DEPARTURE_DELAY": "float64",
    "DEPARTURE_TIME": "float64",
    "ELAPSED_TIME": "float64",
    "SCHEDULED_TIME": "float64",
    "TAXI_IN": "float64",
    "TAXI_OUT": "float64",
    "WHEELS_OFF": "float64",
    "WHEELS_ON": "float64",
}

# Load dataset with correct dtypes
ddf = dd.read_csv("flights.csv", dtype=dtype_fix, assume_missing=True)

# Peek at first few rows
ddf.head()


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015.0,1.0,1.0,4.0,AS,98.0,N407AS,ANC,SEA,5.0,...,408.0,-22.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
1,2015.0,1.0,1.0,4.0,AA,2336.0,N3KUAA,LAX,PBI,10.0,...,741.0,-9.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
2,2015.0,1.0,1.0,4.0,US,840.0,N171US,SFO,CLT,20.0,...,811.0,5.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
3,2015.0,1.0,1.0,4.0,AA,258.0,N3HYAA,LAX,MIA,20.0,...,756.0,-9.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
4,2015.0,1.0,1.0,4.0,AS,135.0,N527AS,SEA,ANC,25.0,...,259.0,-21.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN


In [11]:
#Dataset Information
ddf.info()


<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 31 entries, YEAR to WEATHER_DELAY
dtypes: float64(26), string(5)

In [12]:
#Check Columns

ddf.columns


Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

In [13]:
# Number of Partitions
ddf.npartitions


9

In [14]:
#Convert to pandas for preview
df_sample = ddf.head(10)
df_sample


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015.0,1.0,1.0,4.0,AS,98.0,N407AS,ANC,SEA,5.0,...,408.0,-22.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
1,2015.0,1.0,1.0,4.0,AA,2336.0,N3KUAA,LAX,PBI,10.0,...,741.0,-9.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
2,2015.0,1.0,1.0,4.0,US,840.0,N171US,SFO,CLT,20.0,...,811.0,5.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
3,2015.0,1.0,1.0,4.0,AA,258.0,N3HYAA,LAX,MIA,20.0,...,756.0,-9.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
4,2015.0,1.0,1.0,4.0,AS,135.0,N527AS,SEA,ANC,25.0,...,259.0,-21.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
5,2015.0,1.0,1.0,4.0,DL,806.0,N3730B,SFO,MSP,25.0,...,610.0,8.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
6,2015.0,1.0,1.0,4.0,NK,612.0,N635NK,LAS,MSP,25.0,...,509.0,-17.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
7,2015.0,1.0,1.0,4.0,US,2013.0,N584UW,LAX,CLT,30.0,...,753.0,-10.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
8,2015.0,1.0,1.0,4.0,AA,1112.0,N3LAAA,SFO,DFW,30.0,...,532.0,-13.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN
9,2015.0,1.0,1.0,4.0,DL,1173.0,N826DN,LAS,ATL,30.0,...,656.0,-15.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN


In [15]:
#Describe statistics
ddf.describe().compute()


,YEAR,MONTH,DAY,DAY_OF_WEEK,FLIGHT_NUMBER,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
count,5819079.0,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.819079e+06,5.732926e+06,5.732926e+06,5.730032e+06,5.730032e+06,...,5.819079e+06,5.726566e+06,5.714008e+06,5.819079e+06,5.819079e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06,1.063439e+06
mean,2015.0,6.524085e+00,1.570459e+01,3.926941e+00,2.173093e+03,1.329602e+03,1.335204e+03,9.370158e+00,1.607166e+01,1.357171e+03,...,1.493808e+03,1.476491e+03,4.407057e+00,2.609863e-03,1.544643e-02,1.348057e+01,7.615387e-02,1.896955e+01,2.347284e+01,2.915290e+00
std,0.0,3.405137e+00,8.783425e+00,1.988845e+00,1.757064e+03,4.837518e+02,4.964233e+02,3.708094e+01,8.895574e+00,4.980094e+02,...,5.071647e+02,5.263197e+02,3.927130e+01,5.102012e-02,1.233201e-01,2.800368e+01,2.143460e+00,4.816164e+01,4.319702e+01,2.043334e+01
min,2015.0,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-8.200000e+01,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,-8.700000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2015.0,3.000000e+00,1.100000e+01,2.000000e+00,7.560000e+02,9.250000e+02,9.300000e+02,-4.000000e+00,1.100000e+01,9.450000e+02,...,1.123000e+03,1.117000e+03,-1.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2015.0,6.000000e+00,2.000000e+01,4.000000e+00,1.735000e+03,1.330000e+03,1.336000e+03,-1.000000e+00,1.400000e+01,1.349000e+03,...,1.525000e+03,1.522000e+03,-3.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00,3.000000e+00,8.000000e+00,0.000000e+00
75%,2015.0,9.000000e+00,2.600000e+01,6.000000e+00,3.485000e+03,1.739000e+03,1.751000e+03,1.300000e+01,1.900000e+01,1.805000e+03,...,1.922000e+03,1.922000e+03,1.300000e+01,0.000000e+00,0.000000e+00,1.900000e+01,0.000000e+00,1.900000e+01,3.500000e+01,0.000000e+00
max,2015.0,1.200000e+01,3.100000e+01,7.000000e+00,9.855000e+03,2.359000e+03,2.400000e+03,1.988000e+03,2.250000e+02,2.400000e+03,...,2.400000e+03,2.400000e+03,1.971000e+03,1.000000e+00,1.000000e+00,1.134000e+03,5.730000e+02,1.971000e+03,1.331000e+03,1.211000e+03


In [16]:
#Count rows
ddf.shape[0].compute()


5819079

In [17]:
#Count null values
ddf.isnull().sum().compute()


,0
YEAR,0
MONTH,0
DAY,0
DAY_OF_WEEK,0
AIRLINE,0
FLIGHT_NUMBER,0
TAIL_NUMBER,14721
ORIGIN_AIRPORT,0
DESTINATION_AIRPORT,0
SCHEDULED_DEPARTURE,0


In [19]:
#Flights per day
# Create a DATE column in YYYY-MM-DD format
ddf = ddf.assign(
    DATE = ddf["YEAR"].astype(str) + "-" +
           ddf["MONTH"].astype(str).str.zfill(2) + "-" +
           ddf["DAY"].astype(str).str.zfill(2)
)

# Now group by DATE and count flights
flights_per_day = ddf.groupby("DATE")["FLIGHT_NUMBER"].count().compute()
print(flights_per_day.head())


DATE
2015.0-1.0-1.0     13950
2015.0-1.0-10.0    12344
2015.0-1.0-11.0    14754
2015.0-1.0-12.0    15888
2015.0-1.0-13.0    15180
Name: FLIGHT_NUMBER, dtype: int64


In [20]:
#Plot flights per day
hvplot.extension('bokeh')

ddf.groupby("DATE")["FLIGHT_NUMBER"].count().hvplot.line(
    title="Flights per Day", xlabel="Date", ylabel="Count", width=800, height=400
)



:Curve   [DATE]   (FLIGHT_NUMBER)

In [21]:
#Cancelled flights per day
ddf.groupby("DATE")["CANCELLED"].sum().compute()


,CANCELLED
DATE,
2015.0-1.0-1.0,466.0
2015.0-1.0-10.0,190.0
2015.0-1.0-11.0,354.0
2015.0-1.0-12.0,436.0
2015.0-1.0-13.0,197.0
...,...
2015.0-12.0-5.0,62.0
2015.0-12.0-6.0,49.0
2015.0-12.0-7.0,178.0


In [22]:
#Plot cancelled flights per day
ddf.groupby("DATE")["CANCELLED"].sum().hvplot.line(
    title="Cancelled Flights per Day", xlabel="Date", ylabel="Cancelled"
)


:Curve   [DATE]   (CANCELLED)

In [24]:
#Departure delays per airline
# Departure delays per airline
ddf.groupby("AIRLINE")["DEPARTURE_DELAY"].mean().compute()



,DEPARTURE_DELAY
AIRLINE,
AA,8.900856
AS,1.785801
B6,11.514353
DL,7.369254
EV,8.715934
F9,13.350858
HA,0.485713
MQ,10.125188
NK,15.944766


In [26]:
#Plot departure delays per airline
# Plot departure delays per airline
ddf.groupby("AIRLINE")["DEPARTURE_DELAY"].mean().hvplot.bar(
    title="Average Departure Delay per Airline",
    xlabel="Airline",
    ylabel="Minutes"
)


:Bars   [AIRLINE]   (DEPARTURE_DELAY)

In [28]:
#Arrival delays per airline
ddf.groupby("AIRLINE")["ARRIVAL_DELAY"].mean().compute()


,ARRIVAL_DELAY
AIRLINE,
AA,3.451372
AS,-0.976563
B6,6.677861
DL,0.186754
EV,6.585379
F9,12.504706
HA,2.023093
MQ,6.457873
NK,14.471800


In [29]:
#Persist dataset
ddf_p = ddf.persist()
ddf_p


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
npartitions=9,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,string,float64,string,string,string,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [30]:
#] Count dep delays with persist
ddf_p.groupby("DATE")["DEPARTURE_DELAY"].count().compute()


,DEPARTURE_DELAY
DATE,
2015.0-1.0-1.0,13490
2015.0-1.0-10.0,12161
2015.0-1.0-11.0,14415
2015.0-1.0-12.0,15473
2015.0-1.0-13.0,14994
...,...
2015.0-12.0-5.0,12139
2015.0-12.0-6.0,15059
2015.0-12.0-7.0,15868


In [31]:
#Count arr delays with persist
ddf_p.groupby("DATE")["ARRIVAL_DELAY"].count().compute()


,ARRIVAL_DELAY
DATE,
2015.0-1.0-1.0,13464
2015.0-1.0-10.0,12138
2015.0-1.0-11.0,14339
2015.0-1.0-12.0,15399
2015.0-1.0-13.0,14957
...,...
2015.0-12.0-5.0,12123
2015.0-12.0-6.0,15028
2015.0-12.0-7.0,15800


In [32]:
# Repartition dataset
ddf_re = ddf.repartition(npartitions=10).persist()
ddf_re


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
npartitions=10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float64,float64,float64,float64,string,float64,string,string,string,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,string,float64,float64,float64,float64,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [33]:
# Flights per day (repartitioned)
ddf_re.groupby("DATE")["FLIGHT_NUMBER"].count().compute()


,FLIGHT_NUMBER
DATE,
2015.0-1.0-1.0,13950
2015.0-1.0-10.0,12344
2015.0-1.0-11.0,14754
2015.0-1.0-12.0,15888
2015.0-1.0-13.0,15180
...,...
2015.0-12.0-27.0,16509
2015.0-12.0-28.0,16312
2015.0-12.0-29.0,16199


In [34]:
# Unique carriers per day
ddf.groupby("DATE")["AIRLINE"].nunique().compute()


,AIRLINE
DATE,
2015.0-1.0-4.0,14
2015.0-1.0-5.0,14
2015.0-1.0-15.0,14
2015.0-1.0-23.0,14
2015.0-2.0-5.0,14
...,...
2015.0-11.0-22.0,13
2015.0-11.0-23.0,13
2015.0-11.0-25.0,13


In [36]:
# Plot unique carriers per day
ddf.groupby("DATE")["AIRLINE"].nunique().hvplot.line(
    title="Unique Carriers per Day", xlabel="Date", ylabel="Count"
)



:Curve   [DATE]   (AIRLINE)

In [37]:
# Negative delay handling
f = lambda x: 0 if x < 0 else x
ddf["DEPARTURE_DELAY"].apply(f, meta=pd.Series(dtype="float64")).head()


,DEPARTURE_DELAY
0,0
1,0
2,0
3,0
4,0


In [38]:
# Apply with compute
ddf["DEPARTURE_DELAY"].apply(f, meta=pd.Series(dtype="float64")).mean().compute()


np.float64(12.114237999932321)

In [39]:
# ] Correlation between delays
ddf[["DEPARTURE_DELAY","ARRIVAL_DELAY"]].corr().compute()


,DEPARTURE_DELAY,ARRIVAL_DELAY
DEPARTURE_DELAY,1.000000,0.944672
ARRIVAL_DELAY,0.944672,1.000000


In [40]:
# Top 10 airports with max dep delay
ddf.groupby("ORIGIN_AIRPORT")["DEPARTURE_DELAY"].mean().nlargest(10).compute()



,DEPARTURE_DELAY
ORIGIN_AIRPORT,
14222,89.111111
ILG,29.391753
13964,28.843750
MVY,25.907317
HYA,23.182927
10154,22.857143
10581,20.111111
STC,18.692308
OTH,17.777358


In [41]:
# Top 10 airports with max arr delay
ddf.groupby("DESTINATION_AIRPORT")["ARRIVAL_DELAY"].mean().nlargest(10).compute()


,ARRIVAL_DELAY
DESTINATION_AIRPORT,
13964,26.906250
STC,23.025974
ILG,21.989474
14025,18.000000
TTN,17.433901
GUM,17.320242
OTH,16.740602
MQT,16.152027
ASE,15.829448


In [42]:
# Plot delays by airport
ddf.groupby("ORIGIN_AIRPORT")["DEPARTURE_DELAY"].mean().nlargest(10).hvplot.bar(
    title="Top 10 Origin Airports by Departure Delay"
)


:Bars   [ORIGIN_AIRPORT]   (DEPARTURE_DELAY)

In [43]:
# Save a smaller sample
ddf_sample = ddf.head(20000, compute=True)
ddf_sample.to_csv("airline_small.csv", index=False)


In [45]:
# Reload small dataset
import dask.dataframe as dd

ddf_small = dd.read_csv(
    "airline_small.csv",
    dtype={'CANCELLATION_REASON': 'object'}
)

ddf_small.head()



,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DATE
0,2015.0,1.0,1.0,4.0,AS,98.0,N407AS,ANC,SEA,5.0,...,-22.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,2015.0-1.0-1.0
1,2015.0,1.0,1.0,4.0,AA,2336.0,N3KUAA,LAX,PBI,10.0,...,-9.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,2015.0-1.0-1.0
2,2015.0,1.0,1.0,4.0,US,840.0,N171US,SFO,CLT,20.0,...,5.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,2015.0-1.0-1.0
3,2015.0,1.0,1.0,4.0,AA,258.0,N3HYAA,LAX,MIA,20.0,...,-9.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,2015.0-1.0-1.0
4,2015.0,1.0,1.0,4.0,AS,135.0,N527AS,SEA,ANC,25.0,...,-21.0,0.0,0.0,<NA>,NaN,NaN,NaN,NaN,NaN,2015.0-1.0-1.0


In [48]:
# Final Visualization
ddf.groupby("DATE")[["DEPARTURE_DELAY","ARRIVAL_DELAY"]].mean().hvplot.line(
    title="Daily Average Delays", xlabel="Date", ylabel="Minutes"
)


:NdOverlay   [Variable]
   :Curve   [DATE]   (value)